### Data
We have dataset for Russian market only (here https://www.kaggle.com/c/sberbank-russian-housing-market/), but if you want to apply this methods to Armenian Housing Market, you should do some additional routine work. For example, use selenium module to scrape data from https://www.estate.am/en .

### Methods
Obviously, data contains some categorical features, which should be transformed into appropriate format. Use feature selection method "mutual information regression" and others, if you want.
Data contains some dependencies similar to linear, for example, house_price(house_square) so, as I said, you can help to tree ensemble by pre-exclusion of linear components. For example, use stacking of two models: regularized linear regression and some tree ensemble. Linear regression will take responsibility for linear components, so remaining task without linear dependencies will be much easier for tree ensemble. You can stack more models and create second-level predictor, which make predictions using combination of zero-level raw features and first-level models predictions. Use sklearn pipeline for combining your transformations into directed acyclic graph and train the whole graph at once (see *resources)

### Loss
Optimize mean squared log error, use cross-validation

In [2]:
import pandas as pd
import numpy as np
import  tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
from keras import regularizers
from sklearn.model_selection import GridSearchCV
import xgboost
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import ElasticNet

In [3]:
# Load in the train and test datasets
train = pd.read_csv('C:/Users/Gugo_ML/Downloads/all/train.csv')
test = pd.read_csv('C:/Users/Gugo_ML/Downloads/all/test.csv')

price_doc = train['price_doc']
del train['price_doc']
data = pd.concat([train, test], sort=False)

In [4]:
train.head(5)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,40,9,4,0,13,22,1,0,52,4
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,36,15,3,0,15,29,1,10,66,14
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,25,10,3,0,11,27,0,4,67,10
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,15,11,2,1,4,4,0,0,26,3
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,552,319,108,17,135,236,2,91,195,14


In [5]:
test.head(5)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,8,0,0,0,1,10,1,0,14,1
1,30475,2015-07-01,79.2,NaN,8,17,1,0.0,3,1.0,...,4,1,1,0,2,11,0,1,12,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,42,11,4,0,10,21,0,10,71,11
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,1,1,2,0,0,10,0,0,2,0
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,5,1,1,0,2,12,0,1,11,1


In [6]:
test_id = test['id']
del train['id']
del test['id']
del data['id'] 

In [7]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['day'] = data['timestamp'].dt.day
data['month'] = data['timestamp'].dt.month
data['year'] = data['timestamp'].dt.year
del data['timestamp']

In [8]:
data = data.dropna(axis=1)

In [9]:
data.head(5)

,full_sq,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_education_centers_raion,children_school,school_education_centers_raion,...,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,day,month,year
0,43.0,Bibirevo,6.407578e+06,155572,0.189727,0.000070,9576,5,10309,5,...,0,13,22,1,0,52,4,20,8,2011
1,34.0,Nagatinskij Zaton,9.589337e+06,115352,0.372602,0.049637,6880,5,7759,8,...,0,15,29,1,10,66,14,23,8,2011
2,43.0,Tekstil'shhiki,4.808270e+06,101708,0.112560,0.118537,5879,4,6207,7,...,0,11,27,0,4,67,10,27,8,2011
3,89.0,Mitino,1.258354e+07,178473,0.194703,0.069753,13087,9,13670,10,...,1,4,4,0,0,26,3,1,9,2011
4,77.0,Basmannoe,8.398461e+06,108171,0.015234,0.037316,5706,7,6748,9,...,17,135,236,2,91,195,14,5,9,2011


In [10]:
train.head(5)

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,40,9,4,0,13,22,1,0,52,4
1,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,36,15,3,0,15,29,1,10,66,14
2,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,25,10,3,0,11,27,0,4,67,10
3,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,15,11,2,1,4,4,0,0,26,3
4,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,552,319,108,17,135,236,2,91,195,14


In [11]:
data.dtypes

full_sq                                  float64
sub_area                                  object
area_m                                   float64
raion_popul                                int64
green_zone_part                          float64
indust_part                              float64
children_preschool                         int64
preschool_education_centers_raion          int64
children_school                            int64
school_education_centers_raion             int64
school_education_centers_top_20_raion      int64
healthcare_centers_raion                   int64
university_top_20_raion                    int64
sport_objects_raion                        int64
additional_education_raion                 int64
culture_objects_top_25                    object
culture_objects_top_25_raion               int64
shopping_centers_raion                     int64
office_raion                               int64
thermal_power_plant_raion                 object
incineration_raion  

In [12]:
label_encoder = preprocessing.LabelEncoder()
obj_columns = data.select_dtypes(include='object').columns
for col in obj_columns:
    data[col] = label_encoder.fit_transform(data[col])

In [13]:
data.head(5)

,full_sq,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_education_centers_raion,children_school,school_education_centers_raion,...,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,day,month,year
0,43.0,9,6.407578e+06,155572,0.189727,0.000070,9576,5,10309,5,...,0,13,22,1,0,52,4,20,8,2011
1,34.0,70,9.589337e+06,115352,0.372602,0.049637,6880,5,7759,8,...,0,15,29,1,10,66,14,23,8,2011
2,43.0,129,4.808270e+06,101708,0.112560,0.118537,5879,4,6207,7,...,0,11,27,0,4,67,10,27,8,2011
3,89.0,65,1.258354e+07,178473,0.194703,0.069753,13087,9,13670,10,...,1,4,4,0,0,26,3,1,9,2011
4,77.0,6,8.398461e+06,108171,0.015234,0.037316,5706,7,6748,9,...,17,135,236,2,91,195,14,5,9,2011


In [14]:
data.shape

(38133, 239)

In [15]:
cor = data.corr().abs()
cor

,full_sq,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_education_centers_raion,children_school,school_education_centers_raion,...,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,day,month,year
full_sq,1.000000,0.015449,0.048159,0.032921,0.025658,0.049457,0.019388,0.006026,0.017997,0.006328,...,0.047830,0.032327,0.033964,0.024393,0.035395,0.008517,0.035354,0.001144,0.006046,0.011942
sub_area,0.015449,1.000000,0.175674,0.249431,0.097717,0.112975,0.237644,0.259557,0.231853,0.271138,...,0.061849,0.034697,0.040996,0.036066,0.065493,0.091416,0.118437,0.002128,0.014407,0.027030
area_m,0.048159,0.175674,1.000000,0.402616,0.504642,0.247028,0.343931,0.413345,0.351892,0.422239,...,0.154384,0.221096,0.212935,0.112146,0.194375,0.417230,0.450680,0.004626,0.032697,0.030399
raion_popul,0.032921,0.249431,0.402616,1.000000,0.222023,0.029403,0.953810,0.851239,0.956830,0.828727,...,0.018972,0.037342,0.043717,0.014086,0.014811,0.278420,0.412222,0.009675,0.039740,0.000765
green_zone_part,0.025658,0.097717,0.504642,0.222023,1.000000,0.524211,0.231885,0.278224,0.236786,0.280298,...,0.230933,0.289278,0.276787,0.290717,0.268618,0.371241,0.351514,0.000722,0.025058,0.037310
indust_part,0.049457,0.112975,0.247028,0.029403,0.524211,1.000000,0.035282,0.020991,0.008547,0.005723,...,0.138878,0.079679,0.103414,0.078664,0.121864,0.004103,0.095791,0.005254,0.021615,0.000265
children_preschool,0.019388,0.237644,0.343931,0.953810,0.231885,0.035282,1.000000,0.851009,0.986596,0.827325,...,0.040392,0.001000,0.000673,0.027599,0.024622,0.193584,0.302116,0.008572,0.035436,0.000657
preschool_education_centers_raion,0.006026,0.259557,0.413345,0.851239,0.278224,0.020991,0.851009,1.000000,0.839291,0.957937,...,0.016305,0.042646,0.045042,0.044273,0.039830,0.209946,0.333921,0.009196,0.039663,0.022537
children_school,0.017997,0.231853,0.351892,0.956830,0.236786,0.008547,0.986596,0.839291,1.000000,0.817163,...,0.008450,0.056029,0.055581,0.018063,0.029146,0.240284,0.338702,0.009071,0.032696,0.000108
school_education_centers_raion,0.006328,0.271138,0.422239,0.828727,0.280298,0.005723,0.827325,0.957937,0.817163,1.000000,...,0.039954,0.095023,0.099572,0.015812,0.080976,0.285019,0.421085,0.008703,0.039426,0.013964


In [16]:
upper = cor.where(np.triu(np.ones(cor.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
for i in range(len(to_drop)):
    del data[to_drop[i]]

In [17]:
data.shape

(38133, 143)

In [18]:
#Creating model

x_train = data[:train.shape[0]]
x_test = data[train.shape[0]:]
y_train = price_doc.values

In [19]:
linear_regression = GridSearchCV(estimator=ElasticNet(max_iter=1500), param_grid={'alpha':[2, 1.5], 'l1_ratio':[0.5, 0.7]}, cv=10)
linear_regression.fit(x_train,y_train)

linear_regression_predicts = linear_regression.predict(x_test)
linear_regression_predicts[linear_regression_predicts<0]=np.mean(linear_regression_predicts)
linear_regression_train_predicts = linear_regression.predict(x_train)

C:\Users\Gugo_ML\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\Gugo_ML\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\Gugo_ML\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\Gugo_ML\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want t

C:\Users\Gugo_ML\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\Gugo_ML\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\Gugo_ML\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\Gugo_ML\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want t

In [20]:
linear_regression.best_params_

{'alpha': 1.5, 'l1_ratio': 0.7}

In [21]:
xgbr = GridSearchCV(XGBRegressor(n_jobs=4), param_grid={'reg_alpha':[0.001, 0.01],'min_child_weight':[2, 3],
              'max_depth':[7, 8],'reg_lambda':[100, 150],
             'n_estimators':[200, 250]}, cv=10)
xgbr.fit(x_train, y_train, eval_metric='rmse')

xgbr_predicts = xgbr.predict(x_test)
xgbr_train_predicts = xgbr.predict(x_train)

In [22]:
xgbr.best_params_

{'max_depth': 7,
 'min_child_weight': 2,
 'n_estimators': 250,
 'reg_alpha': 0.01,
 'reg_lambda': 150}

In [23]:
x_train = np.column_stack([linear_regression_train_predicts, xgbr_train_predicts])
x_test = np.column_stack([linear_regression_predicts, xgbr_predicts])

In [27]:
seed = 7
tf.set_random_seed(seed)
np.random.seed(seed)

model = keras.Sequential()
model.add(keras.layers.Dense(units=64, activation='softplus',
                bias_initializer=keras.initializers.RandomUniform(minval=0.001, maxval=0.01, seed=seed),
                kernel_regularizer=regularizers.l1_l2(),
                bias_regularizer=regularizers.l1_l2(),
                input_shape=(x_train.shape[1],)))
model.add(keras.layers.Dense(units=64, activation='softplus',
                bias_initializer=keras.initializers.RandomUniform(minval=0.001, maxval=0.01, seed=seed),
                kernel_regularizer=regularizers.l1_l2(),
                bias_regularizer=regularizers.l1_l2()))
model.add(keras.layers.Dense(units=32, activation='softplus',
                bias_initializer=keras.initializers.RandomUniform(minval=0.001, maxval=0.01, seed=seed),
                kernel_regularizer=regularizers.l1_l2(),
                bias_regularizer=regularizers.l1_l2()))
model.add(keras.layers.Dense(units=16, activation='softplus',
                bias_initializer=keras.initializers.RandomUniform(minval=0.001, maxval=0.01, seed=seed),
                kernel_regularizer=regularizers.l1_l2(),
                bias_regularizer=regularizers.l1_l2()))
model.add(keras.layers.Dense(units=16, activation='softplus',
                bias_initializer=keras.initializers.RandomUniform(minval=0.001, maxval=0.01, seed=seed),
                kernel_regularizer=regularizers.l1_l2(),
                bias_regularizer=regularizers.l1_l2()))
model.add(keras.layers.Dense(units=1, activation='linear',
                bias_initializer=keras.initializers.RandomUniform(minval=0.001, maxval=0.01, seed=seed)))

model.compile(loss='mean_squared_logarithmic_error',
              optimizer='adam',
              metrics=['msle'])

In [28]:
model.fit(x_train, y_train, epochs=100, validation_split=0.3)

Train on 21329 samples, validate on 9142 samples
Epoch 1/100
21329/21329 [==============================] - 4s 187us/step - loss: 4.2806 - mean_squared_logarithmic_error: 0.9962 - val_loss: 2.2016 - val_mean_squared_logarithmic_error: 0.1859
Epoch 2/100
21329/21329 [==============================] - 2s 81us/step - loss: 2.0579 - mean_squared_logarithmic_error: 0.2348 - val_loss: 1.8316 - val_mean_squared_logarithmic_error: 0.1831
Epoch 3/100
21329/21329 [==============================] - 1s 69us/step - loss: 1.7294 - mean_squared_logarithmic_error: 0.2296 - val_loss: 1.5329 - val_mean_squared_logarithmic_error: 0.1791
Epoch 4/100
21329/21329 [==============================] - 2s 71us/step - loss: 1.4435 - mean_squared_logarithmic_error: 0.2248 - val_loss: 1.2572 - val_mean_squared_logarithmic_error: 0.1723
Epoch 5/100
21329/21329 [==============================] - 2s 73us/step - loss: 1.1822 - mean_squared_logarithmic_error: 0.2198 - val_loss: 1.0162 - val_mean_squared_logarithmic_erro

21329/21329 [==============================] - 1s 55us/step - loss: 0.2632 - mean_squared_logarithmic_error: 0.1966 - val_loss: 0.2126 - val_mean_squared_logarithmic_error: 0.1462
Epoch 44/100
21329/21329 [==============================] - 1s 54us/step - loss: 0.2632 - mean_squared_logarithmic_error: 0.1967 - val_loss: 0.2163 - val_mean_squared_logarithmic_error: 0.1507
Epoch 45/100
21329/21329 [==============================] - 1s 55us/step - loss: 0.2628 - mean_squared_logarithmic_error: 0.1967 - val_loss: 0.2115 - val_mean_squared_logarithmic_error: 0.1450
Epoch 46/100
21329/21329 [==============================] - 1s 55us/step - loss: 0.2626 - mean_squared_logarithmic_error: 0.1965 - val_loss: 0.2115 - val_mean_squared_logarithmic_error: 0.1451
Epoch 47/100
21329/21329 [==============================] - 1s 55us/step - loss: 0.2631 - mean_squared_logarithmic_error: 0.1970 - val_loss: 0.2158 - val_mean_squared_logarithmic_error: 0.1468
Epoch 48/100
21329/21329 [======================

21329/21329 [==============================] - 1s 61us/step - loss: 0.2753 - mean_squared_logarithmic_error: 0.1953 - val_loss: 0.2229 - val_mean_squared_logarithmic_error: 0.1442
Epoch 86/100
21329/21329 [==============================] - 1s 51us/step - loss: 0.2724 - mean_squared_logarithmic_error: 0.1949 - val_loss: 0.2207 - val_mean_squared_logarithmic_error: 0.1438
Epoch 87/100
21329/21329 [==============================] - 1s 50us/step - loss: 0.2709 - mean_squared_logarithmic_error: 0.1946 - val_loss: 0.2195 - val_mean_squared_logarithmic_error: 0.1438
Epoch 88/100
21329/21329 [==============================] - 2s 78us/step - loss: 0.2699 - mean_squared_logarithmic_error: 0.1949 - val_loss: 0.2179 - val_mean_squared_logarithmic_error: 0.1435
Epoch 89/100
21329/21329 [==============================] - 2s 75us/step - loss: 0.2680 - mean_squared_logarithmic_error: 0.1947 - val_loss: 0.2160 - val_mean_squared_logarithmic_error: 0.1436
Epoch 90/100
21329/21329 [======================

In [30]:
model_predictions = model.predict(x_test).flatten()

In [34]:
submission = pd.DataFrame(np.column_stack([np.arange(30474, 38136), model_predictions]), columns=['id', 'price_doc'])
submission['id'] = submission['id'].astype('int64')
submission.to_csv('submission.csv', index=False)